In [148]:
import re
import copy
import math
import random

# Adjacent Vacant Status

In [149]:
# Find the adjacency of each cell in the grid
def calc_adj(grid):
    M, N = len(grid), len(grid[0])
    adj_mat = copy.deepcopy(grid)

    for i in range(M):
        for j in range(N):
            if grid[i][j] != '':
                adj_mat[i][j] = -1
            else:
                c = 0
                for dx, dy in [(0, -1), (-1, 0), (0, 1), (1, 0)]:
                    x, y = i, j
                    x += dx
                    y += dy
                    if 0 <= x < M and 0 <= y < N and grid[x][y] == '':
                        c += 1
                adj_mat[i][j] = c

    return adj_mat

# if a qubit has a vacant neighbor, returns the neighbor with highest adjacency
def has_adj_vacant(grid, qubit, adjMat, max_adj):
    M, N = len(grid), len(grid[0])
    maxadj_qubit = None
    for i, j in [(0, -1), (-1, 0), (0, 1), (1, 0)]:
        x, y = qubit
        x += i
        y += j
        if 0 <= x < M and 0 <= y < N:
            if grid[x][y] == '':
                if max_adj == -1 or max_adj < adjMat[x][y]:
                    max_adj = adjMat[x][y]
                    maxadj_qubit = (x, y)
    return maxadj_qubit, max_adj

# Placing Qubits on the Grid

In [150]:
def place_qubits(grid, qubit_priority, adjMat):
    M, N = len(grid), len(grid[0])
    i, j = M//2, N//2
    placed = []

    # Put the highest priority qubit in the middle
    if grid[i][j] == '':
        grid[i][j] = qubit_priority.pop(0)
        placed.append((i, j))
        adjMat = calc_adj(grid)

    while qubit_priority:
        # Find the next cell with highest adjacency adjacent to the already placed qubits
        max_adj = -1
        for x, y in placed:
            maxadj_qubit, new_max = has_adj_vacant(grid, (x, y), adjMat, max_adj)
            if maxadj_qubit and (max_adj == -1 or max_adj < new_max):
                i, j = maxadj_qubit
                max_adj = new_max
        if max_adj == -1:
            break
        # Place the next qubit in the cell with highest adjacency
        grid[i][j] = qubit_priority.pop(0)
        placed.append((i, j))
        adjMat = calc_adj(grid)
        # print(grid, adjMat, placed, qubit_priority)

    return grid, adjMat

# Utility Functions

In [151]:
def find_qubit(grid, qubit):
    M, N = len(grid), len(grid[0])
    for i in range(M):
        for j in range(N):
            if grid[i][j] == qubit:
                return (i, j)
    return None

def check_adjescent(grid, qubit1, qubit2):
    x1, y1 = qubit1
    x2, y2 = qubit2
    if x1 == x2 and abs(y1-y2) == 1:
        return True
    if y1 == y2 and abs(x1-x2) == 1:
        return True
    return False

def is_valid_cell(grid, x, y):
    M, N = len(grid), len(grid[0])
    return 0 <= x < M and 0 <= y < N and grid[x][y] != ''

def find_weight(src, dest, weights):
    for i in weights:
        if (i[0],i[1]) == (src,dest) or (i[0],i[1]) == (src,dest):
            return i[2]
    return 0

# Path Finding in Grid

In [152]:
# Check if cell(x,y) is in the grid
def is_valid_cell(grid, x, y):
    M, N = len(grid), len(grid[0])
    return 0 <= x < M and 0 <= y < N and grid[x][y] != ''

# Find all possible paths from src to dest using BFS
def find_path_utils(grid, src, dest, visited, path, paths):
    x, y = src
    visited[x][y] = True
    path.append(src)
    if src == dest:
        paths.append(copy.deepcopy(path))
    else:
        for i, j in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            x, y = src
            x += i
            y += j
            if is_valid_cell(grid, x, y) and not visited[x][y]:
                find_path_utils(grid, (x, y), dest, visited, path, paths)
    path.pop()
    visited[x][y] = False

# Find all possible paths from src to dest using BFS
# and return the shortest ones
def find_min_paths(grid, src, dest):
    M, N = len(grid), len(grid[0])
    visited = [[False]*N for _ in range(M)]
    paths = []
    find_path_utils(grid, src, dest, visited, [], paths)

    minpaths, minlen = [], len(paths[0])
    for path in paths:
        if len(path) < minlen:
            minlen = len(path)
    for path in paths:
        if len(path) == minlen:
            minpaths.append(path)
    # print(paths, minpaths)
    return minpaths

# Rearranging qubits on the Grid

In [153]:
def rearrange_qubits(grid, qubit1, qubit2, weights):
    M, N = len(grid), len(grid[0])
    if qubit1 == None or qubit2 == None:
        return grid, 0

    freqs = {}
    for path in find_min_paths(grid, qubit1, qubit2):
        freq = 0
        for i in range(len(path)-1):
            qubit1 = grid[path[i][0]][path[i][1]]
            qubit2 = grid[path[i+1][0]][path[i+1][1]]
            freq += find_weight(qubit1, qubit2, weights)
        freqs[tuple(path)] = freq
    # print(freqs)

    best_path = []
    if freqs:
        max_freq = max(freqs.values())
        best_path = [path for path, freq in freqs.items() if freq == max_freq]
        best_path = random.choice(best_path)
        # print(best_path)

        best_len = len(best_path)
        max_swaps = best_len-2
        gen_combo = lambda s: ((i, s-i) for i in range(s+1))
        ways = list(gen_combo(max_swaps))
        # print(ways)
        
        if len(ways) > 0:
            best_way = random.choice(ways)
            # print(best_way)
        swap_src, swap_dest = best_way

        count = 0
        for i in range(swap_src):
            grid[best_path[i][0]][best_path[i][1]], grid[best_path[i+1][0]][best_path[i+1][1]] = \
                grid[best_path[i+1][0]][best_path[i+1][1]], grid[best_path[i][0]][best_path[i][1]]
            count += 1
        
        j = best_len-1
        for i in range(swap_dest):
            grid[best_path[j][0]][best_path[j][1]], grid[best_path[j-1][0]][best_path[j-1][1]] = \
                grid[best_path[j-1][0]][best_path[j-1][1]], grid[best_path[j][0]][best_path[j][1]]
            j -= 1
            count += 1
        
        count = int(math.ceil(0.8*count))

    return grid, count

### Read the input file and find preference table

In [154]:
# filename = input("Enter the filename: ")
filename = 'rd32-v0_67'
with open(f'benchmarks/revlib_decomposed/{filename}.real', 'r') as f:
    data = f.read()

qubits = int(re.search(r".numvars ([\d]*)", data).group(1))
gates = re.search(r"(?s)(?<=.begin).*?(?=.end)", data, re.MULTILINE).group(0)
lines = gates.strip().split('\n')

# print(qubits, lines)
temp = [[0]*qubits for _ in range(qubits)]
interacting_qubits = []

for t, line in enumerate(lines, 1):
    params = line.split(' ')
    if len(params) == 2: continue
    
    qubit1 = int(re.search(r"\d+", params[1]).group(0))
    qubit2 = int(re.search(r"\d+", params[2]).group(0))
    interacting_qubits.append((qubit1, qubit2))
    
    i, j = min(qubit1, qubit2), max(qubit1, qubit2)
    temp[i-1][j-1] += 1/t
print(interacting_qubits)

count = [0]*qubits
weights = []
for i in range(qubits):
    for j in range(qubits):
        if temp[i][j] != 0:
            # coeff = (len(interaction_times[i]) * len(interaction_times[j]))/len(lines)
            # print(i+1, j+1, coeff, temp[i][j])
            weights.append([i+1, j+1, temp[i][j]]) # Can check removing this coeff
            count[i] += 1
            count[j] += 1
print(f"Graph: {weights}")
print(f"Count: {count}")

[(2, 4), (1, 4), (1, 2), (2, 4), (3, 4), (2, 4), (2, 3), (3, 4)]
Graph: [[1, 2, 0.3333333333333333], [1, 4, 0.5], [2, 3, 0.14285714285714285], [2, 4, 1.4166666666666667], [3, 4, 0.325]]
Count: [2, 3, 2, 3]


# Qubit Placement and Swap Counts

In [155]:
qubit_priority = [count.index(max(count))+1]
while len(qubit_priority) < qubits:
    max_weight = -1
    for i in range(qubits):
        if i+1 not in qubit_priority:
            weight = 0
            for j in qubit_priority:
                weight += find_weight(i+1, j, weights)
            if weight > max_weight:
                max_weight = weight
                next_qubit = i+1
    qubit_priority.append(next_qubit)
print(f"Qubit priority: {qubit_priority}")

# M, N = map(int, input("Enter the grid size: ").split())
M, N = 3, 2
grid = [['']*N for _ in range(M)]
adjMat = calc_adj(grid)
grid, adjMat = place_qubits(grid, qubit_priority, adjMat)
print(grid, adjMat)

best_grid, min_swap = [], 1000
for i in range(10000):
    temp_grid = copy.deepcopy(grid)
    swap_count = 0
    for t, interaction in enumerate(interacting_qubits, 1):
        if len(interaction) == 2:
            qubit1, qubit2 = interaction
            x1, y1 = find_qubit(temp_grid, qubit1)
            x2, y2 = find_qubit(temp_grid, qubit2)

            if not check_adjescent(temp_grid, (x1, y1), (x2, y2)):
                # print(f"\nswap at line {t}: {interaction}")
                temp_grid, swap = rearrange_qubits(temp_grid, (x1, y1), (x2, y2), weights)
                swap_count += swap
                # print(grid)

    # print("\nGrid: ",grid)
    # print(f"swap count: {swap_count}")
    if swap_count < min_swap:
        min_swap = swap_count
        best_grid = grid
        print(f"min swap count: {min_swap} at iteration {i}")

Qubit priority: [2, 1, 3, 4]
[['', 3], [1, 2], ['', 4]] [[0, -1], [-1, -1], [0, -1]]
min swap count: 6 at iteration 0
min swap count: 5 at iteration 1
min swap count: 4 at iteration 2
min swap count: 3 at iteration 16
